In [ ]:
import pandas as pd

# Define the two DataFrames
df1 = pd.DataFrame([['a', 1], ['b', 2]],
                   columns=['letter', 'number'])
df2 = pd.DataFrame([['c', 1], ['d', 2]],
                   columns=['letter', 'number'])

# Concatenate the DataFrames vertically (axis=0) and reset the index
# The default axis is 0, so we don't need to specify it.
df_combined = pd.concat([df1, df2], ignore_index=True)

# Print the resulting DataFrame and its index type
print("Combined DataFrame:")
print(df_combined)
print("\nIndex Type:")
print(df_combined.index)

In [ ]:
import pandas as pd

# Define the first DataFrame (df1)
df1_dict = {
        'id': ['1', '2', '3', '4', '5'],
        'Feature1': ['A', 'C', 'E', 'G', 'I'],
        'Feature2': ['B', 'D', 'F', 'H', 'J']}
df1 = pd.DataFrame(df1_dict, columns = ['id', 'Feature1', 'Feature2'])

# Define the second DataFrame (df2)
df2_dict = {
        'id': ['1', '2', '6', '7', '8'],
        'Feature1': ['K', 'M', 'O', 'Q', 'S'],
        'Feature2': ['L', 'N', 'P', 'R', 'T']}
df2 = pd.DataFrame(df2_dict, columns = ['id', 'Feature1', 'Feature2'])

print("--- DataFrame 1 ---")
print(df1)
print("\n--- DataFrame 2 ---")
print(df2)
print("\n" + "="*50 + "\n")

In [ ]:
# Inner Merge: Keeps only the intersecting IDs ('1' and '2')
df_inner = pd.merge(df1, df2, on='id', how='inner')

print("Output 1: Inner Merge")
print(df_inner)

In [ ]:
# Outer Merge: Keeps all IDs from both DataFrames
df_outer = pd.merge(
    df1,
    df2,
    on='id',
    how='outer',
    suffixes=('_df1', '_df2') # Set custom suffixes for clarity
)

# Replace 'nan' with 'nannan' for visual match with your example (optional step)
df_outer = df_outer.fillna('nan')

print("\nOutput 2: Full Outer Merge")
print(df_outer)

In [9]:
import pandas as pd
import numpy as np

# generate days
all_dates = pd.date_range('2021-01-01', '2021-12-15')
business_dates = pd.bdate_range('2021-01-01', '2021-12-31')

# generate tickers
tickers = ['AAPL', 'FB', 'GE', 'AMZN', 'DAI']

# create indexs
index_alt = pd.MultiIndex.from_product([all_dates, tickers], names=['Date', 'Ticker'])
index = pd.MultiIndex.from_product([business_dates, tickers], names=['Date', 'Ticker'])

# create DFs
market_data = pd.DataFrame(index=index,
                        data=np.random.randn(len(index), 3),
                        columns=['Open','Close','Close_Adjusted'])

alternative_data = pd.DataFrame(index=index_alt,
                                data=np.random.randn(len(index_alt), 2),
                                columns=['Twitter','Reddit'])

# --- Question 1: Merge MultiIndex ---
merged_df = market_data.merge(alternative_data, 
                             how='left', 
                             left_index=True, 
                             right_index=True)

# --- Question 2: Fill Missing Values ---
filled_df = merged_df.fillna(0)

# Validation check:
validation_result = filled_df.sum().sum() == merged_df.sum().sum()

# Output the required validation results
print(f"DataFrame Shape: {merged_df.shape}")
print("\nmerged_df.head():")
print(merged_df.head().to_markdown())
print(f"\nValidation Check (Q2): {validation_result}")

DataFrame Shape: (1305, 5)

merged_df.head():
|                                            |      Open |     Close |   Close_Adjusted |   Twitter |    Reddit |
|:-------------------------------------------|----------:|----------:|-----------------:|----------:|----------:|
| (Timestamp('2021-01-01 00:00:00'), 'AAPL') | -0.710692 |  0.683618 |        -0.734952 |  1.29918  | -0.915968 |
| (Timestamp('2021-01-01 00:00:00'), 'FB')   | -0.90291  |  1.76793  |         0.96074  | -1.54523  |  0.803316 |
| (Timestamp('2021-01-01 00:00:00'), 'GE')   | -1.86151  | -0.764768 |         0.242463 | -1.64624  |  1.21953  |
| (Timestamp('2021-01-01 00:00:00'), 'AMZN') |  0.323755 | -1.49049  |         1.15857  | -0.904395 | -1.10329  |
| (Timestamp('2021-01-01 00:00:00'), 'DAI')  |  0.825224 |  0.481902 |         1.35673  |  0.248462 | -1.03814  |

Validation Check (Q2): True


In [15]:
import pandas as pd
import numpy as np

def winsorize(df: pd.DataFrame, quantiles: list) -> pd.DataFrame:
    """
    Applies Winsorization to all numerical columns in a DataFrame without a for loop.
    
    Args:
        df (pd.DataFrame): The DataFrame to process.
        quantiles (list): A list containing the lower and upper quantiles (e.g., [0.20, 0.80]).
        
    Returns:
        pd.DataFrame: The DataFrame with winsorized values.
    """
    if len(quantiles) != 2 or quantiles[0] >= quantiles[1]:
        raise ValueError("Quantiles must be a list of two values: [lower_q, upper_q], where lower_q < upper_q.")
    
    lower_q, upper_q = quantiles[0], quantiles[1]
    df_winsorized = df.copy()

    # 1. Select only the numerical columns to process
    numeric_df = df.select_dtypes(include=np.number)
    
    # 2. Calculate the cut-off values (bounds) across ALL numerical data.
    # We flatten the numerical data to a 1D array before calculating percentiles.
    series_data = numeric_df.values.flatten()
    
    # Use numpy.percentile to calculate the cut-off values (requires 0-100 range)
    lower_bound = np.percentile(series_data, lower_q * 100)
    upper_bound = np.percentile(series_data, upper_q * 100)
    
    # 3. Apply the clip method to the ENTIRE numerical DataFrame at once
    # This replaces the need for the loop.
    clipped_data = numeric_df.clip(lower=lower_bound, upper=upper_bound, axis=1)
    
    # 4. Update the numerical columns in the copy
    df_winsorized[numeric_df.columns] = clipped_data
        
    return df_winsorized

# --- Verification ---
df = pd.DataFrame(range(1,11), columns=['sequence'])

# Running the function:
print(winsorize(df, [0.20, 0.80]).to_markdown())

|    |   sequence |
|---:|-----------:|
|  0 |        2.8 |
|  1 |        2.8 |
|  2 |        3   |
|  3 |        4   |
|  4 |        5   |
|  5 |        6   |
|  6 |        7   |
|  7 |        8   |
|  8 |        8.2 |
|  9 |        8.2 |


In [16]:
import pandas as pd
import numpy as np

# --- Reuse the Winsorize Function ---
def winsorize(df: pd.DataFrame, quantiles: list) -> pd.DataFrame:
    # ... (Your winsorize function body remains the same) ...
    if len(quantiles) != 2 or quantiles[0] >= quantiles[1]:
        raise ValueError("Quantiles must be a list of two values: [lower_q, upper_q], where lower_q < upper_q.")
    
    lower_q, upper_q = quantiles[0], quantiles[1]
    df_winsorized = df.copy()

    for col in df.select_dtypes(include=np.number).columns:
        series = df[col]
        lower_bound = np.percentile(series.dropna(), lower_q * 100)
        upper_bound = np.percentile(series.dropna(), upper_q * 100)
        df_winsorized[col] = series.clip(lower=lower_bound, upper=upper_bound)
        
    return df_winsorized

# --- Generate the Data Set ---
groups = np.concatenate([np.ones(10), np.ones(10)+1,  np.ones(10)+2, np.ones(10)+3, np.ones(10)+4])

df = pd.DataFrame(data= zip(groups,
                            range(1,51)),
                columns=["group", "sequence"])

# --- Apply Groupby and Winsorize (FIXED) ---
quantiles_to_use = [0.05, 0.95]

# FIX: Add include_groups=False to silence the FutureWarning
winsorized_df = df.groupby('group', group_keys=False).apply(
    lambda x: winsorize(x, quantiles=quantiles_to_use),
    include_groups=False # <-- This is the fix
)

# NOTE: group_keys=False is also added to prevent the group key from becoming part of the index.

# Since we dropped the grouping column ('group'), we need to re-merge it or use the original index
winsorized_df = winsorized_df.reset_index(drop=True)[['sequence']]

# --- Verification ---
print("Winsorized DataFrame (First Rows):")
print(winsorized_df.head(11).to_markdown())

Winsorized DataFrame (First Rows):
|    |   sequence |
|---:|-----------:|
|  0 |       1.45 |
|  1 |       2    |
|  2 |       3    |
|  3 |       4    |
|  4 |       5    |
|  5 |       6    |
|  6 |       7    |
|  7 |       8    |
|  8 |       9    |
|  9 |       9.55 |
| 10 |      11.45 |
